In [1]:
import numpy as np
import pandas as pd
import math
import json
import sys
import time
from InduceC45 import c45, readFiles
from classifier import classify, evaluate, traverseTree

In [2]:
def sampleDf(df, attrs, m, k):
    datapoints = df.sample(frac=k)
    newdata = datapoints.iloc[:,:-1].sample(m,axis=1).join(datapoints.iloc[:,-1])
    
    newattrs = {x: attrs[x] for x in newdata.iloc[:,:-1].columns}
    return newdata, newattrs

In [3]:
def createRandomForest(df, attrs, m, k, n, thresh=0.2):
    trees=[]
    for i in range(n):
        sample, sampleAttrs = sampleDf(df,attrs,m,k)
        trees.append(c45(sample, sampleAttrs, thresh))
    return trees

In [15]:
def classifyForest(df, attrs, forest):
    preds = []
    for i, row in df.iterrows():
        predsmini = []
        for tree in forest:
            keys = list(tree)
            predsmini.append(traverseTree(row, tree[keys[-1]], keys[-1]))
        preds.append([i, pd.Series(predsmini).mode()[0]])
    return pd.DataFrame(preds, columns=['index', 'prediction']).set_index('index')

In [17]:
if __name__ == "__main__":
    thresh=0.2
#     if len(sys.argv) == 6:
#         _, datafile, m, k, N, outputfile = sys.argv
#     else if len(sys.argv) == 7:
#         _, datafile, m, k, N, outputfile, thresh = sys.argv
#     else:
#         print("Usage: python3 randomForest.py <datafile.csv> <m> <k> <N> <outputFileName.csv> [thresh=0.2]")
#         exit(1)
    
    
    m = 2
    k = .2
    n = 10
    datafile = "./data/iris.data.csv"
    outputfile = "./output/iris.forest.preds"
    
    df, filename, isLabeled, attrs = readFiles(datafile)
    
    forest = createRandomForest(df,attrs,m,k,n,thresh)
    preds = classifyForest(df, attrs, forest)
    results = evaluate(df,preds)
    
#     preds = classifySimple(df,c45(df,attrs,thresh))
#     results = evaluate(df,preds)
    for v in results:
        print(v, results[v])
#     print(createRandomForest(df,attrs,m,k,n,thresh))

accuracy 0.98
errorRate 0.02
numClassified 150
numCorrect 147
numErrors 3
confusionLabel Actual ↓, Predicted →
confusion                  Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa             50.0              0.0             0.0
Iris-versicolor          0.0             48.0             1.0
Iris-virginica           0.0              2.0            49.0
results     sepalLength  sepalWidth  petalLength  petalWidth         species  \
2           5.1         3.5          1.4         0.2     Iris-setosa   
3           4.9         3.0          1.4         0.2     Iris-setosa   
4           4.7         3.2          1.3         0.2     Iris-setosa   
5           4.6         3.1          1.5         0.2     Iris-setosa   
6           5.0         3.6          1.4         0.2     Iris-setosa   
..          ...         ...          ...         ...             ...   
147         6.7         3.0          5.2         2.3  Iris-virginica   
148         6.3         2.5          5.0       